### Chains
`Chains` are one of the most powerful features in LangChain, allowing you to combine multiple components into cohesive workflows. This section presents two different methodologies for implementing chains - the traditional `SequentialChain` approach and the newer LangChain Expression Language (`LCEL`).

**Why Chains Matter:**

Chains solve a fundamental problem with LLMs. Chains are primarily designed to handle a single prompt and generate a single response. However, most real-world applications require multi-step reasoning, accessing different tools, or breaking complex tasks into manageable pieces. Chains allow you to orchestrate these complex workflows.

**Evolution of Chain Patterns:**

Traditional chains (`LLMChain`, `SequentialChain`) were LangChain's first implementation, offering a structured but somewhat rigid approach. LCEL (using the pipe operator `|`) represents a more flexible, functional approach that's easier to compose and debug.

**Note:** While both approaches are presented here for educational purposes, **LCEL is the recommended pattern for new development.** The SequentialChain approach continues to be supported for backward compatibility, but the LangChain community has largely transitioned to the LCEL pattern for its superior flexibility and expressiveness.

#### **Simple Chain**
#### Traditional Approach: LLMChain
Here is a simple single chain using `LLMChain`.

In [25]:
from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

chat_llm = ChatOllama(
    model = "llama3",
    temperature = 0.8 ,
    num_predict = 256,
)

template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}
 YOUR RESPONSE:
"""
prompt_template = PromptTemplate(template = template, input_variables = ['location'])

location_chain = LLMChain(llm = chat_llm , prompt=prompt_template, output_key='meal')

res = location_chain.invoke(input={'location':'China'}) 
print(res['meal'])

For China, I would suggest the classic dish "Kung Pao Chicken" () from Sichuan Province. This spicy stir-fry dish is made with marinated chicken, peanuts, vegetables, and chili peppers in a savory sauce, served over steamed rice. The combination of flavors and textures is a perfect representation of Chinese cuisine's bold and harmonious balance.

Would you like me to provide the recipe?


#### Modern Approach: LCEL

Here is the same chain implemented using the more modern LCEL (LangChain Expression Language) approach with the pipe operator:

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}
 YOUR RESPONSE:
"""

prompt = PromptTemplate.from_template(template)

location_chain_lcel = prompt | chat_llm | StrOutputParser() 

result = location_chain_lcel.invoke({"location":"China"})

print(result)

China!

One of the most iconic and beloved dishes in China is Peking Duck (). This dish originated in Beijing (formerly known as Peking) and has been a staple of Chinese cuisine for centuries. The dish is characterized by its crispy-skinned, melt-in-your-mouth duck that's served with pancakes, scallions, and hoisin sauce.

To make Peking Duck, the duck is first puffed up to create a layer of fat under the skin, which is then scored to create a beautiful, intricate pattern. The duck is then roasted in a special oven at high heat until the skin is crispy and golden brown. Meanwhile, pancakes are steamed to be soft and pliable, and scallions and hoisin sauce are prepared on the side.

When served, the diner wraps slices of the Peking Duck in a pancake with a dollop of hoisin sauce and a sprinkle of scallions. It's a truly unique and unforgettable culinary experience!


#### **Simple sequential chain**

Sequential chains allow you to use output of one LLM as the input for another LLM. This approach is beneficial for dividing tasks and maintaining the focus of your LLM.

In this example, you see a sequence that:

- Gets a meal from a location
- Gets a recipe for that meal
- Estimates the cooking time for that recipe

This pattern is incredibly valuable for breaking down complex tasks into logical steps, where each step depends on the output of the previous step. The traditional approach uses `SequentialChain`, while the modern `LCEL` approach uses piping and `RunnablePassthrough.assign`.


#### Traditional Approach: `SequentialChain`

In [30]:
from langchain_classic.chains import SequentialChain

template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.
 YOUR RESPONSE:
"""

prompt_template = PromptTemplate(template=template, input_variables=['meal'])

dish_chain = LLMChain(llm=chat_llm, prompt=prompt_template, output_key='recipe')

template = """Given the recipe {recipe}, estimate how much time I need to cook it.
 YOUR RESPONSE:
"""

prompt_template = PromptTemplate(template=template, input_variables=['recipe'])


recipe_chain = LLMChain(llm=chat_llm, prompt=prompt_template, output_key='time')

overall_chain = SequentialChain(
    chains = [location_chain,dish_chain,recipe_chain],
    input_variables = ['location'],
    output_variables = ['meal','recipe','time'],
    verbose = True
)
from pprint import pprint
pprint(overall_chain.invoke(input={'location':'China'}))



> Entering new SequentialChain chain...

> Finished chain.
{'location': 'China',
 'meal': 'China!\n'
         '\n'
         "In China, I'd like to suggest a classic dish from the Sichuan "
         'province: Kung Pao Chicken ().\n'
         '\n'
         'This spicy stir-fry is made with marinated chicken, peanuts, '
         'vegetables (such as bell peppers and scallions), chili peppers, and '
         'Sichuan peppercorns. The combination of crispy chicken, crunchy '
         'peanuts, and numbingly spicy flavors has made Kung Pao Chicken a '
         'beloved dish around the world.\n'
         '\n'
         'Would you like me to share a simple recipe for this iconic Chinese '
         'dish?',
 'recipe': "What a delicious choice! I'd love to help you with a simple recipe "
           "for Kung Pao Chicken. Here's a straightforward version to get you "
           'started:\n'
           '\n'
           '**Serves 4**\n'
           '\n'
           '**Ingredients:**\n'
           '\

In [31]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

location_template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}

YOUR RESPONSE:
"""

dish_template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.

YOUR RESPONSE:
"""

time_template = """Given the recipe {recipe}, estimate how much time I need to cook it.

YOUR RESPONSE:
"""
location_chain_lcel = (
    PromptTemplate.from_template(location_template)  
    | chat_llm                                  
    | StrOutputParser()                         
)
dish_chain_lcel = (
    PromptTemplate.from_template(dish_template)      
    | chat_llm                                    
    | StrOutputParser()                       
)
time_chain_lcel = (
    PromptTemplate.from_template(time_template)      
    | chat_llm                                    
    | StrOutputParser()                             
)

overall_chain_lcel = (
    
    RunnablePassthrough.assign(meal=lambda x: location_chain_lcel.invoke({"location": x["location"]}))
   
    | RunnablePassthrough.assign(recipe=lambda x: dish_chain_lcel.invoke({"meal": x["meal"]}))
    
    | RunnablePassthrough.assign(time=lambda x: time_chain_lcel.invoke({"recipe": x["recipe"]}))
)
result = overall_chain_lcel.invoke({"location": "China"})
pprint(result)

{'location': 'China',
 'meal': 'China!\n'
         '\n'
         'One of the most iconic and beloved dishes from China is... Peking '
         'Duck! \n'
         '\n'
         'Peking duck, also known as Beijing roast duck, is a classic Chinese '
         "dish originating from Beijing. It's a must-try when visiting China "
         'or trying authentic Chinese cuisine.\n'
         '\n'
         'The dish involves roasting a whole duck over an open flame, creating '
         'a crispy skin and tender meat. The duck is typically served with '
         'pancakes, scallions, and hoisin sauce, allowing each guest to '
         'assemble their own perfect bite.\n'
         '\n'
         "Peking duck is often considered the national dish of China, and it's "
         'no wonder why - its rich flavors and textures have captured the '
         'hearts (and stomachs) of people around the world!',
 'recipe': "I'm excited to share this iconic Chinese dish with you!\n"
           '\n'
           